In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import chemiscope
from widget_code_input import WidgetCodeInput
from ipywidgets import Textarea
from iam_utils import *
import ase
from ase.io import read, write
from ase.calculators import lj, eam
from ase.optimize import BFGS, LBFGS

In [ ]:
#### AVOID folding of output cell 

In [ ]:
%%html

<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:4000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [ ]:
data_dump = WidgetDataDumper(prefix="module_05")
display(data_dump)

In [ ]:
module_summary = Textarea("general comments on this module", layout=Layout(width="100%"))
data_dump.register_field("module-summary", module_summary, "value")
display(module_summary)

_Reference textbook / figure credits: 
Charles Kittel, "Introduction to solid-state physics", Chapters 20-21; 
[Helmut Föll, Defects in Crystals](https://www.tf.uni-kiel.de/matwis/amat/def_en/index.html) 
Wikipedia_

# Point defects

Point defects are the simplest kind of imperfections that can be found in the structure of a crystalline material. _Vacancies_ are empty lattice sites, _interstitials_ are additional atoms that fill a void in the structure.  Impurities can also occur in the form of _substitutional_ atoms, that replace some of the atoms of the main lattice. Defects can have a large impact on the electronic properties of a material, drive transport of ions, and are often related to solid-state transformations. 

Particulary for ionic crystals, where defects must preserve  overall charge neutrality, defects must come in pairs: _Frenkel defects_ are a vacancy-interstitial pair (conceptually corresponding to an atom moved from a lattice to an interstitial site), _Schottky defects_ are paris of vacancies formed by atoms with opposite charge. 

<img src="figures/defecttypes.png" width="600"/>

In the vast majority of cases the presence of a defect results in the *increase* of the energy of the crystal by an amount $\Delta E$. Then why are there defects in solids? The answer involves a combination of thermodynamics and kinetics. You may recall that the stability of a system at constant temperature $T$ and pressure is determined by its *Gibbs free energy* $G=H-TS$, where $H$ is the enthalpy (which to a very good approximation is equal **approximation or equal? (@michele)** to the internal energy for a solid at ambient pressure) minus a term that involves the *entropy* of the system. 

A full discussion of the microscopic definition of entropy is beyond the scope of this module, but we will just say that one can define it as $S=k_B \ln \Omega$, where $k_B=1.38\cdot 10^{-23}$ J/K, and $\Omega$ is the number of microscopic configurations that are consistent with the macroscopic observables and boundary conditions. For a lattice with $N$ sites and $M$ vacancies, one can estimate $\Omega$ by combinatorial arguments

$$
\ln \Omega = \ln \frac{N!}{M!(N-M)!} \approx N \ln N - M\ln M -(N-M)\ln (N-M)
$$

where we also used Stirling approximation $\ln N! \approx N \ln N$. 

By ignoring constant terms that depend only on $N$, and rewriting in terms of the vacancy concentration $x=M/N$, one can find $N^{-1}\ln\Omega \approx -x\ln x -(1-x)\ln(1-x)$. Thus, the molar free energy $G/N$ of a solid with a concentration of vacancies $x$ can be written as 

$$
g(x) =  x \Delta E  + k_B T\left[ x\ln x + (1-x)\ln(1-x)\right]
$$

<span style="color:blue">**01a** The equilibrium concentration of vacancies can be obtained differentiating the expression and solving for $x$. Write the analytical expression (you do not need to write down the steps, for this question the expression is enough). What is the highest _equilibrium_ molar vacancy concentration that is theoretically possible?</span>

In [ ]:
ex1a_txt = Textarea("Write the answer here.", layout=Layout(width="100%"))
data_dump.register_field("ex1a-answer", ex1a_txt, "value")
display(ex1a_txt)

In [ ]:
# Boltzman constant in eV/K
kB = 8.617333262145e-5

def plot_molar_free_energy(axes, deltaE, T):
    # x = M/N
    x = np.linspace(0.001, 0.999, 100)
    
    molar_free_energy_fun = lambda x: x*deltaE+kB*T*(x*np.log(x)+(1-x)*np.log(1-x))
    molar_free_energies = molar_free_energy_fun(x)
    equilibrium_vacancy_concentration_at_T = 1/(np.exp(deltaE/(kB*T)) + 1)
    molar_free_energies_at_equilibrium = molar_free_energy_fun(equilibrium_vacancy_concentration_at_T)
    
    temperatures = np.linspace(10, 4000, 80)
    equilibrium_vacancy_concentrations = 1/(np.exp(deltaE/(kB*temperatures)) + 1)

    axes[0].clear()
    axes[0].plot(x, molar_free_energies)
    axes[0].scatter(equilibrium_vacancy_concentration_at_T, molar_free_energies_at_equilibrium, color='red', s=80, edgecolors='black')
    axes[0].set_title('')
    axes[0].set_xlim(0.001,0.999)
    axes[0].set_ylim(-0.3,0.5)
    axes[0].set_xlabel("Vacancy concentration $x$")
    axes[0].set_ylabel("Molar free energy $g(x)$ / eV")


    axes[1].clear()
    axes[1].plot(temperatures, equilibrium_vacancy_concentrations, c='r')
    axes[1].scatter(T, equilibrium_vacancy_concentration_at_T, color='red', s=80, edgecolors='black')
    axes[1].set_title('')
    axes[1].set_xlim(0,4000)
    axes[1].set_ylim(0,0.5)
    axes[1].set_xlabel("Temperature $T$ / K")
    axes[1].set_ylabel("Equilibrium vacancy concentration")

In [ ]:
fig_ax = plt.subplots(1, 2, figsize=(8,3.8), tight_layout=True)

mfe_pb = WidgetParbox( deltaE=(0.2, 0.01, 0.3, 0.01, r"$\Delta E\, /\, eV$"), 
                       T=(2000, 300, 4000, 50, r"$T\, /\, K$ "))

ex01_wp = WidgetPlot(plot_molar_free_energy, mfe_pb, fig_ax=fig_ax)
display(ex01_wp)

The widgets above show the molar free energy of the defective crystal as a function of vacancy concentration for the temperature and vacancy formation energy selected in the sliders. The right-hand panel shows the equilibrium concentration _as a function of temperature_ (i.e. the red point on the free-energy curve determines the value of the concentration on the right-hand plot for that value of $T$. Manipulating this widget and observing the qualitative and quantitative behavior, answer the following questions.

<span style="color:blue"> **01b** What is the general trend of the equilibrium vacancy concentration with respect to temperature? How does $\Delta E$ influence this trend?</span>

In [ ]:
ex1b_txt = Textarea("Write the answer here.", layout=Layout(width="100%"))
data_dump.register_field("ex1b-answer", ex1b_txt, "value")
display(ex1b_txt)

<span style="color:blue">**02** Do you think that it would be possible, in practice, to reach the maximum equilibrium vacancy concentration? What might happen before that? </span>

In [ ]:
ex2_txt = Textarea("Write the answer here", layout=Layout(width="100%"))
data_dump.register_field("ex2-answer", ex2_txt, "value")
display(ex2_txt)

<span style="color:blue">**03** Metals at room temperature often have a much higher vacancy concentration than it would be predicted by the expressions above. How can you explain this observation? </span>

In [ ]:
ex3_txt = Textarea("Write the answer here", layout=Layout(width="100%"))
data_dump.register_field("ex3-answer", ex3_txt, "value")
display(ex3_txt)

# Vacancy formation energy

In order to compute the vacancy formation energy, we need first to compute the energy of an atom in a perfect crystal. If the calculation is performed for a supercell with $n$ atoms, the molar energy is the total potential for the cell divided by $n$, $\epsilon = E_n / n$. 


One then needs to build a supercell (that could contain a different number $m$ of atoms) and remove one. The energy of a vacancy can be computed by subtracting from the energy of the supercell that of $m-1$ atoms of a perfect crystal, 

$$ 
\Delta E = E_{m-1} - (m-1)\epsilon
$$

The visualizer below shows an _fcc_ structure, with one vacancy. Can you see what atom is missing? The second frame in the viewer shows green points at the ideal _fcc_ lattice positions, to help you identify the vacancy. 

In [ ]:
from ase.lattice.cubic import FaceCenteredCubic
import copy

al_fcc = FaceCenteredCubic(size=(2,2,2), symbol='Al', pbc=True)
ref_f = al_fcc.copy(); ref_f.symbols = "F"*len(ref_f)
al_fcc_vacancy = copy.deepcopy(al_fcc)

del al_fcc_vacancy[17]
al_fcc_strucs = [al_fcc, al_fcc_vacancy]



cs = chemiscope.show([al_fcc_vacancy, al_fcc_vacancy+ref_f], mode="structure",                      
                     settings={"structure":[{"bonds":False, "unitCell":True,"supercell":{"0":1,"1":1,"2":1},
                                            "environments": {"cutoff": 40}}]}                    
                    )

display(cs)


<span style="color:blue">**04** Write a function that computes the vacancy formation energy of aluminum (EAM potential), using a conventional _fcc_ cell (4-atoms, lattice parameter $a_0 = 4.05$Å) to get the molar energy, and a supercell replicated _nrep_ times in each direction, from which you can then remove one atom. Return the vacancy formation energy computed from the formula above. </span>

**not so clear what $n$ to use to compute the molar energy. Even we let the student figure out that it converges with n large enough, or we fix it. I think fixing it makes correction easier and it is also not so interesting for the student to figure out that the energy converges with supercell size (@michele)**

**For me it was initially not clear that $\Delta E$ does not depend on the choice of the deleted atom. Is this because it has can be expressed with a unit cell with a basis size of one? But doesn't it mean that for a different lattices this formula does not work? Maybe we could make this a question?(@michele)**

_NB: you can remove an atom from an `ase.Atoms` structure by calling `del structure[[index]]` (note the double brackets). See the [module on interatomic potentials](./04-Potentials.ipynb) to remind you how the ASE potential calculator works_

In [ ]:
ex04_wci = WidgetCodeInput(
        function_name="vacancy_builder", 
        function_parameters="nrep",
        docstring="""
Builds a model with a nrep×nrep×nrep supercell and a vacancy, and returns the vacancy formation energy.
:return: energy
""",
        function_body="""
import numpy as np
from ase import Atoms
from ase.calculators import eam

return 0 # <-- remove this after having completed the code below
calc =  ...  # initializes the calculator 

# this is an fcc unit cell with orientations along (111) (1-10) (11-2)
a0 = 4.05 # lattice parameter of Al
# constructs a unit cell
fcc_cell = Atoms("Al4", cell=..., positions=..., pbc=True, calculator=calc ) 
e_cell = ...

# creates the supercell
vacancy = fcc_cell.replicate( ... )
vacancy.calc = calc
# removes an atom
...
e_supercell = ...

# compute formation energy combining e_cell and e_supercell
e_vacancy = ...
# print(nrep, e_vacancy) <- if you want to see the precise value printed out, uncomment this line
return e_vacancy
"""
        )

def plot_vac(ax):
    func = ex04_wci.get_function_object() 
    values = np.asarray([ [n, func(n)] for n in [1,2,3,4]])
    
    ax.plot(values[:, 0], values[:,1], 'b*')
    
    ax.set_xlabel(r"$n_{\mathrm{rep}}$")
    ax.set_ylabel(r"$E_{\mathrm{vac}}$ / eV")
    #ax.set_ylim([-3, 3])
    
ex04_plot = WidgetPlot(plot_vac)

data_dump.register_field("ex4-function", ex04_wci, "function_body")        
ex04_wcc = WidgetCodeCheck(ex04_wci, ref_values = {(1,) : 1.0199205493799237, (2,) : 0.770153142090478}, demo=ex04_plot)       
display(ex04_wcc)

<span style="color:blue">**05** What is the vacancy formation energy? Does the estimated value change with cell size? Does it depend on the index of the atom that is removed? Comment on what you observe. </span>

In [ ]:
ex5_txt = Textarea("Write the answer here", layout=Layout(width="100%"))
data_dump.register_field("ex5-answer", ex5_txt, "value")
display(ex5_txt)

# Energy relaxation

The exercise above assumes that atoms in the vicinity of a vacancy remain in their ideal lattice position. This is obviously a harsh approximation: with the change in environment, atoms will rearrange to find a more favorable position. This will both change the structure, and the energy of the system, and therefore affect the vacancy formation energy. 

To account for this rearrangement, one can _relax_ the geometry of the vacancy structure. The idea is simply to minimize the potential energy $V$ of the system, looking for a structure for which the force $\mathbf{f}=-\nabla V$ is zero. There is a large number of schemes that have been proposed to achieve this goal (see here those that are [implemented in ASE](https://wiki.fysik.dtu.dk/ase/ase/optimize.html)) which underscores the importance of energy relaxation to compute accurately the stability of structures. 

The most naïve approach, called [steepest descent](https://en.wikipedia.org/wiki/Gradient_descent) iteratively optimizes the structure following the gradient, 

$$
\mathbf{r} \leftarrow \mathbf{r} -\alpha \nabla V(\mathbf{r}).
$$

Even though it may seem to be an efficient idea (following the direction of maximum decrease of the potential) in a high-dimensional problem it leads to very slow convergence. If you are curious, you can read about the mathematical aspects of optimization, and why steepest descent does not work, in these excellent [lecture notes](https://www.cs.cmu.edu/~quake-papers/painless-conjugate-gradient.pdf). 

Here we will use a rather sophisticated method, named [BFGS](https://en.wikipedia.org/wiki/Broyden%E2%80%93Fletcher%E2%80%93Goldfarb%E2%80%93Shanno_algorithm) from the names of its inventors. 
In practice, a BFGS optimization can be run as follows:

```python
from ase.optimize import BFGS
structure = Atoms( ... , calculator=...)   # you can also initialize the structure and set the calculator with structure.calc = ...
opt = BFGS(structure)
opt.run(fmax=0.01)
```


We will start by looking at a pre-coded function that minimizes the energy of a Lennard-Jones dimer. 

In [ ]:
#AG: write a stub of a function, that save the optimization trajectory to a file. Use ase.opt.BFGS 
# The demo should load and visualize that file in chemiscope, with the energy vs iteration as a property panel

In [ ]:
# just some code for debugging
def al_bfgs_opt(sigma, epsilon, rc, force_threshold):
    from ase.optimize import BFGS
    from ase.calculators import lj
    
    ljcalc = lj.LennardJones(sigma=sigma, epsilon=epsilon, rc=rc)
    # referernce to verify lj
    #ljcalc = eam.EAM(potential='data/Al99.eam.alloy')
    structure = ase.Atoms('Al4',
                          cell = [4.05,4.05,4.05],
                          positions=[[0,0,0],[0,2.025,2.025],[2.025,0,2.025],[2.025,2.025,0]],
                          pbc=True)
    # for 2 repetitions not much optimization happens
    structure = structure.repeat(3)
    structure.calc = ljcalc
    del structure[17]
    opt = BFGS(structure, trajectory='al_bfgs.traj')
    opt.max_steps=10
    opt.run(fmax=force_threshold)
al_bfgs_opt(2.,1,5,0.1) 


In [ ]:
ex06_wci = WidgetCodeInput(
        function_name="optimize_al_with_vacancy", 
        function_parameters="sigma, epsilon, rc, force_threshold",
        docstring="""
Optimizes the geometry of a 3x3x3 supercell of Al fcc with a vacancy.

:param sigma: LJ sigma
:param epsilon: LJ epsilon
:param rc: LJ rc
:param force_threshold: threshold to reach so that the optimizer stops
""",
            function_body="""
import ase 
from ase.optimize import BFGS
from ase.calculators import lj

ljcalc = lj.LennardJones(sigma=sigma, epsilon=epsilon, rc=rc)
# referernce to verify lj
#ljcalc = eam.EAM(potential='data/Al99.eam.alloy')
structure = ase.Atoms('Al4',
                      cell = [4.05,4.05,4.05],
                      positions=[[0,0,0],[0,2.025,2.025],[2.025,0,2.025],[2.025,2.025,0]],
                      pbc=True)
# for 2 repetitions not much optimization happens
# !DO NOT CHANGE NUMBER OF REPETITIONS!
structure = structure.repeat(3)
structure.calc = ljcalc
# !DO NOT CHANGE INDEX OF ATOM!
del structure[55]
# !DO NOT CHANGE NAME OF TRAJECTORY!
opt = BFGS(structure, trajectory='al_bfgs.traj')
opt.max_steps=100
opt.run(fmax=force_threshold)
"""
        )

In [ ]:
# hack for now, not sure if other widget than WidgetPlot, can be used with WidgetParbox
fig_ax = plt.subplots(1, 1, figsize=(0.01,0.01), tight_layout=True)
fig_ax[0].set_visible(False)

# for print output of the code widget
print_output = Output()

hb = VBox(layout=Layout())

def fun_ex06(ax, sigma, epsilon, rc, force_threshold):
    print_output.clear_output()
    hb.children = ()
    with print_output:
        ex06_wci.get_function_object()(sigma, epsilon, rc, force_threshold/100)
    frames = ase.io.read('al_bfgs.traj', ':')
    structure = ase.Atoms('Al4',
                          cell = [4.05,4.05,4.05],
                          positions=[[0,0,0],[0,2.025,2.025],[2.025,0,2.025],[2.025,2.025,0]],
                          pbc=True).repeat(3)
    vacancy_position = structure.positions[55]

    for frame in frames:
        frame += ase.Atom('F', position=vacancy_position)

    # @michele Do you know if one can set the default playback speed?
    cs = chemiscope.show(frames, mode="structure",
                         settings={"structure":[{"bonds":False, "unitCell":False,"supercell":{"0":1,"1":1,"2":1},
                                                "environments": {"cutoff": 40}}]})
    hb.children = (cs,)

# values small as 0.001 cause visual bugs in the parbox which we try to avoid by upscaling the range
ex06_pb = WidgetParbox(
                        sigma=(2.6, 0.1, 5., 0.1, r"$\sigma$"), 
                        epsilon=(0.4, 0.1, 0.5, 0.01, r"$\epsilon$"),
                        rc=(6., 2., 12, 0.1, r"$r_c$"),
                        force_threshold=(1., 0.1, 5., 0.1, r"$\mathbf{f}_\textrm{threshold}\times 10^{2}$")
                     )
ex06_wp = WidgetPlot(fun_ex06, parbox=ex06_pb, fig_ax=fig_ax)


ex06_wcc = WidgetCodeCheck(ex06_wci, ref_values = {}, demo = (ex06_wp, print_output, hb))
display(ex06_wcc)

<span style="color:blue">**06** Read the function above, that takes as arguments the parameters of a LJ potential, initializes a dimer at separation equal to $\sigma$, and runs a geometry optimization.  Observe in the visualizer the behavior of the energy and distance. How many steps are needed to convergence (set $\sigma=2.6, \epsilon=0.4, r_c=6, \mathbf{f}_\textrm{threshold}=0.01$ to ensure reproducibility) ? How many steps are needed if you make the convergence threshold 10 times larger and smaller? </span>

In [ ]:
ex6_txt = Textarea("Write the answer here", layout=Layout(width="100%"))
data_dump.register_field("ex6-answer", ex6_txt, "value")
display(ex6_txt)

<span style="color:blue">**07a** Write a function that computes the vacancy formation energy _after having optimized the geometry of the vacancy_.</span>

_Hint: You can copy much of the code you used in exercise 4. DO NOT CHANGE THE NAME OF THE OUTPUT FILE IN THE OPTIMIZER BECAUSE IT IS USED FOR THE VISUALIZATION_

<span style="color:blue">**07b** What is the vacancy formation energy, after relaxation? Compare the system-size convergence with what you observed in the unrelaxed case. What changed? </span>

In [ ]:
ex7b_txt = Textarea("Write the answer here", layout=Layout(width="100%"))
data_dump.register_field("ex7-answer", ex7b_txt, "value")
display(ex7b_txt)

# Decohesion curves

The surface energy of a material $\gamma$ is the energy that one has to pay to create a unit area of surface. It is an important parameter that has implications for fracture mechanics, tribology, and adhesion. 

<img src="figures/surface-energy-2.png" width="600"/>

In a computational model, one can realize quite easily the thought experiment of achieving a clear-cut cleavage of a crystal along a high-symmetry surface, using exactly the same strategy discussed in [the crystallography module](./02-Crystallography.ipynb) to generate a surface: a bulk structure is formed with a unit cell aligned along appropriate directions, and then one of the lattice parameter is extended, to create a slab geometry, with _two_ surfaces separated by vacuum. 

By computing the energy as a function of separation, we can build _decohesion curves_ that give a cue on the magnitude of the surface energy. The curve is usually expressed such that the energy for the ideal crystal structure is zero, and the energy at large separation corresponds to the surface energy. 

<span style="color:blue">**08a** Modify this function so that it returns a decohesion curve, expressed as a surface energy in eV/Å², and baselined in such a way that the large-separation value corresponds to the surface energy $\gamma$.  </span>

In [ ]:
# AG: the function should already build a large-ish (say 4x4x4) supercell of Al, along the (111,01-1, 211) axes.
# It should get just one option, a drop-down box picking the direction. It should return the energy curve and the frames, 
# and these should be visualized by the demo as a chemiscope. They will only need to subtract the zero and compute the area
# to return things in eV/A^2. add plenty of comments

<span style="color:blue">**08b** What is the surface energy of aluminum along the (111), (01-1), (211) directions? </span>

In [ ]:
ex8_txt = Textarea("Write the answer here", layout=Layout(width="100%"))
data_dump.register_field("ex8-answer", ex8_txt, "value")
display(ex8_txt)

# Dislocations

Dislocations are line defects that can be understood as a loss of register between lattice planes - essentially one section of a plane of atoms is removed, and the remaining pieces are glued together. 
Dislocations can move when a material is subject to strain, and are involved in the mechanism that underlie plastic deformations in metals. The mechanisms by which dislocations are created in a real material can be [quite complicated](https://en.wikipedia.org/wiki/Frank%E2%80%93Read_source).  

<img src="figures/dislocation-edge.png" width="400"/>
<div style='text-align:center; font-style:italic'>
   Credits: Wikipedia, user Magasjukur2, License: CC-BY-SA
</div>


Even considering the idealized scenario in which a lattice plane is removed to form an edge dislocation, the image above is highly unrealistic. Atoms around a dislocation relax to form a distorted structure (_dislocation core_) surrounded by a long-range elastic field, whose form can be evaluated analytically based on continuum elasticity theory. 
A relatively simple example of the complex atomic rearrangements that can occur around a dislocation core is the splitting of a $1/2(1\bar{1}0)\{111\}$ dislocations in an _fcc_ crystal into two _partials_ separated by a stacking fault. The process is illustrated schematically below: a dislocation along the $(1\bar{1}0)$ directions corresponds to removal of _two_ $(110)$ lattice planes. The large deformation of the lattice can be accommodated by forming a pair of partial dislocations (that correspond to a single missing plane each) that are separated by a stacking fault region.

The process is shown schematically in the figure below, and we are going to investigate it with an explicit simulation

<img src="figures/split_disl_perspective.png" width="600"/>
<div style='text-align:center; font-style:italic'>
   Credits: Helmut Föll, Defects in Crystals
</div>

<span style="color:blue">**08a** The function below generates an Al _fcc_ structure. It removes a couple of 9$(110)$ half-planes, generating _a pair_ of dislocations. A distortion is then introduced to "nudge" the atoms towards closing the gap. A geometry optimization is then run to minimize the energy of the system. 
The demo widget shows the process, and the energies as computed by the chosen potential. 
Here you basically only need to understand what the function is doing and - as a minimal tweak - combine the energies computed in the function to return the energies in terms of _line_ energy of the dislocation (energies are in eV, lengths in Å). </span>

In [ ]:
from iam_utils import *

In [ ]:
ex09_wci = WidgetCodeInput(
        function_name="relax_dislocation", 
        function_parameters="",
        docstring="""
Builds and relaxes a dislocation model for a 1/2(1\bar{1}0){111} dislocation
:return: structures, energies 
Structures to be visualized and line energies estimated for the structures along the relaxation
""",
        function_body="""
import numpy as np
from ase import Atoms
from ase.io import read
from ase.calculators import lj, eam
from ase.optimize import LBFGSLineSearch

calc = lj.LennardJones(sigma=2.62, epsilon=0.41, rc=2*2.62)

# this is an fcc unit cell with orientations along (111) (1-10) (11-2)
a0 = 4.04 # lattice parameter of Al
h0 = np.sqrt(np.asarray([[3,0,0],[0, 1/2, 0],[0,0,3/2]]))
pos0 = np.sqrt(np.asarray([[ 0, 0, 0],[0,1/8, 9/24],[1/3,0,16/24],[1/3,1/8,1/24],[4/3,0,4/24],[4/3,1/8,25/24]]))
struc = Atoms("Al6", cell=a0*h0, positions=pos0*a0, pbc=True  )
struc.calc = calc # assigns the calculator
eal6 = struc.get_potential_energy() # gets energy of a perfect unit cell

# creates a supercell
supercell = struc.repeat((6,24,1))
dislocation = supercell.copy()

# removes a slice of atoms (two (110) half-planes)
sel = np.where((np.abs(supercell.positions[:,1]-supercell.cell[1,1]/2-a0*0.12)< a0*0.35+ 1e-5) & 
               (np.abs(supercell.positions[:,0]-supercell.cell[0,0]/2)<supercell.cell[0,0]/4+1e-3) )[0]
del(dislocation[sel])
dislocation.calc = calc
edis = dislocation.get_potential_energy()

# pulls atoms closer at the center of the gap, to facilitate convergence
relax = dislocation.copy()
xx = relax.positions
relax.positions[:,1] -= (
    0.25*a0*np.sign(xx[:,1]-supercell.cell[1,1]/2)*
    np.exp(-(xx[:,1]-supercell.cell[1,1]/2)**2*0.5/(2*a0)**2)*
    np.exp(-(xx[:,0] -supercell.cell[0,0]/2)**2*0.5/((supercell.cell[0,0]/8)**2))
)     

#runs geometry optimization
relax.calc = calc
opt = LBFGSLineSearch(relax, trajectory='dislocation-lj.xyz', memory=50)
opt.run(fmax=0.001)

# reads data from the geop log
opt_traj = read('dislocation-lj.xyz',':')[::10]

# uses dummy F atoms to show the lattice positions in the absence of the dislocation (ideal structure)
supercell.symbols = "F"*len(supercell)
frames=[dislocation+supercell]+[(f+supercell) for f in opt_traj]
energies = np.asarray([edis]+[f.calc.results['energy'] for f in opt_traj])
properties=dict(index=np.arange(1+len(opt_traj)),
                energy=energies)
return frames, properties
"""
        )

def ex09_updater():
    stdout = Output(layout=Layout(width='100%', height='100%', max_height='200px', overflow_y='scroll'))
    vbox = VBox([stdout],layout=Layout(overflow='hidden'))
    display(vbox)
    with stdout:
        frames, properties = ex09_wci.get_function_object()()
    cs = chemiscope.show(frames = frames, properties=properties,
                             settings= {'structure' : [{'bonds': False, 'unitCell': True, 'spaceFilling': False, 'supercell':{'0':1,'1':1,'2':1}, 
                                                           'keepOrientation': True,}] }
                           )
    vbox.children += (cs,)
    

ex09_wcc = WidgetCodeCheck(ex09_wci, ref_values = {},
                           demo=WidgetUpdater(updater=ex09_updater))
display(ex09_wcc)

<span style="color:blue">**08b** What is the line energy for the dissociated dislocation? How many $(110)$ lattice planes separate the two partials?</span>

_Hint: give a rough estimate, this is how the center of a partial dislocation looks like:_
<img src="figures/partial.png"  width="200"/>

In [ ]:
ex9b_txt = Textarea("Write the answer here", layout=Layout(width="100%"))
data_dump.register_field("ex9b-answer", ex9b_txt, "value")
display(ex9b_txt)

<span style="color:blue">**09c** (optional) Modify the function to use the Al EAM potential rather than the LJ one. What is the dislocation line energy? Observe the separation between the partials. How many lattice planes separate them? Compare with the case of the LJ model. </span>

_The EAM is much slower, launch the calculation and go get a coffee..._

In [ ]:
ex9c_txt = Textarea("Write the answer here", layout=Layout(width="100%"))
data_dump.register_field("ex9c-answer", ex9c_txt, "value")
display(ex9c_txt)